# AI Beast - RAG Analysis

This notebook demonstrates RAG (Retrieval-Augmented Generation) operations.

## Features Covered
- Connecting to Qdrant vector store
- Collection analysis
- Document embedding inspection
- Semantic search testing

In [ ]:
# Setup path and imports
import sys
sys.path.insert(0, '/home/jovyan/ai_beast')

import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

## Connect to Qdrant

Connect to the Qdrant vector database for RAG operations.

In [ ]:
from qdrant_client import QdrantClient

# Connect to Qdrant on host
QDRANT_URL = "http://host.docker.internal:6333"

try:
    client = QdrantClient(url=QDRANT_URL)
    collections = client.get_collections()
    print(f"✅ Connected to Qdrant")
    print(f"📚 Collections: {len(collections.collections)}")
except Exception as e:
    print(f"❌ Could not connect to Qdrant: {e}")
    print("Make sure Qdrant is running (beast start qdrant)")

## List Collections

View all available vector collections.

In [ ]:
if 'client' in dir():
    collections = client.get_collections().collections
    
    if collections:
        print("Available Collections:\n")
        for coll in collections:
            info = client.get_collection(coll.name)
            print(f"📁 {coll.name}")
            print(f"   Points: {info.points_count:,}")
            print(f"   Vectors: {info.vectors_count:,}")
            print(f"   Status: {info.status}")
            print()
    else:
        print("No collections found. Run 'beast rag ingest' to create one.")

## Collection Analysis

Analyze a specific collection's contents.

In [ ]:
# Select collection to analyze
COLLECTION_NAME = "ai_beast_docs"  # Change this to your collection

if 'client' in dir():
    try:
        collection_info = client.get_collection(COLLECTION_NAME)
        
        print(f"Collection: {COLLECTION_NAME}")
        print(f"="*50)
        print(f"Points count: {collection_info.points_count:,}")
        print(f"Vectors count: {collection_info.vectors_count:,}")
        print(f"Status: {collection_info.status}")
        
        if collection_info.config:
            config = collection_info.config
            if hasattr(config, 'params'):
                params = config.params
                if hasattr(params, 'vectors'):
                    vectors_config = params.vectors
                    print(f"\nVector Configuration:")
                    if hasattr(vectors_config, 'size'):
                        print(f"  Dimension: {vectors_config.size}")
                    if hasattr(vectors_config, 'distance'):
                        print(f"  Distance: {vectors_config.distance}")
    except Exception as e:
        print(f"Collection '{COLLECTION_NAME}' not found: {e}")
        print("\nAvailable collections:")
        for c in client.get_collections().collections:
            print(f"  - {c.name}")

## Sample Points

View sample documents from the collection.

In [ ]:
if 'client' in dir():
    try:
        # Get sample points
        points, _ = client.scroll(
            collection_name=COLLECTION_NAME,
            limit=5,
            with_payload=True,
            with_vectors=False,
        )
        
        print(f"Sample Documents ({len(points)} points):\n")
        for i, point in enumerate(points, 1):
            print(f"--- Document {i} (ID: {point.id}) ---")
            payload = point.payload or {}
            
            if 'source' in payload:
                print(f"Source: {payload['source']}")
            if 'text' in payload:
                text = payload['text'][:200] + '...' if len(payload.get('text', '')) > 200 else payload.get('text', '')
                print(f"Text: {text}")
            print()
    except Exception as e:
        print(f"Error fetching points: {e}")

## Semantic Search Test

Test semantic search with a sample query.

In [ ]:
# Generate embedding using Ollama
from modules.ollama import OllamaClient

async def get_embedding(text: str, model: str = "nomic-embed-text"):
    """Get embedding from Ollama."""
    async with OllamaClient(base_url="http://host.docker.internal:11434") as client:
        return await client.embed(text, model=model)

# Test query
QUERY = "How do I download a model?"

try:
    embedding = await get_embedding(QUERY)
    print(f"✅ Generated embedding (dim: {len(embedding)})")
except Exception as e:
    print(f"❌ Could not generate embedding: {e}")
    embedding = None

In [ ]:
# Perform semantic search
if 'client' in dir() and embedding:
    try:
        results = client.search(
            collection_name=COLLECTION_NAME,
            query_vector=embedding,
            limit=5,
            with_payload=True,
        )
        
        print(f"Search Results for: '{QUERY}'\n")
        print(f"="*60)
        
        for i, result in enumerate(results, 1):
            print(f"\n{i}. Score: {result.score:.4f}")
            payload = result.payload or {}
            if 'source' in payload:
                print(f"   Source: {payload['source']}")
            if 'text' in payload:
                text = payload['text'][:300] + '...' if len(payload.get('text', '')) > 300 else payload.get('text', '')
                print(f"   Text: {text}")
    except Exception as e:
        print(f"Search failed: {e}")

## Embedding Analysis

Visualize embedding distributions using dimensionality reduction.

In [ ]:
from sklearn.decomposition import PCA

if 'client' in dir():
    try:
        # Get points with vectors
        points, _ = client.scroll(
            collection_name=COLLECTION_NAME,
            limit=100,
            with_payload=True,
            with_vectors=True,
        )
        
        if points:
            # Extract vectors
            vectors = np.array([p.vector for p in points if p.vector is not None])
            
            if len(vectors) > 2:
                # PCA reduction to 2D
                pca = PCA(n_components=2)
                vectors_2d = pca.fit_transform(vectors)
                
                # Plot
                plt.figure(figsize=(10, 8))
                plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.6, c='steelblue')
                plt.title(f'Document Embeddings (PCA) - {COLLECTION_NAME}')
                plt.xlabel('PC1')
                plt.ylabel('PC2')
                plt.tight_layout()
                plt.show()
                
                print(f"Visualized {len(vectors)} embeddings")
                print(f"Explained variance: {pca.explained_variance_ratio_.sum():.2%}")
            else:
                print(f"Need more than 2 vectors for PCA (got {len(vectors)})")
    except Exception as e:
        print(f"Error analyzing embeddings: {e}")